In [21]:
##加载模型
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F
import json
import math
tokenizer = AutoTokenizer.from_pretrained("detected_model")
model = AutoModelForCausalLM.from_pretrained("detected_model", torch_dtype=torch.bfloat16)

In [22]:
#加载数据集
import os
os.environ['TOKENIZERS_PARALLELISM'] = "True"
flatten = lambda l : [x for s in l for x in s]
shuffle = lambda l : random.sample(l, k=len(l))
data_path='dataset/valid.json'
print("loading from json...")
with open(data_path, 'r') as f:
    data = f.read()
    lines=json.loads(data)
sentences=[d['text'] for d in lines]

loading from json...


使用min-K prob 方法 伪代码如下：
1: Input: A sequence of tokens x = x1, x2, ..., xN , decision threshold ϵ
2: Output: Membership of the sequence x
3: for i = 1 to N do
4: Compute − log p(xi|x1, . . . , xi−1)
5: end for
6: Select the top k% of tokens from x with the lowest probability and add to Min-k%(x)
7: MIN-K% PROB(x) = P xi∈Min-k%(x) − log p(xi|x1, ..., xi−1)
8: If MIN-K% PROB(x) > ϵ : return Non-member Else: return Member

In [83]:
#min-k prob
from tqdm import tqdm
import numpy as np

sum_min_k_prob=[]
for j in range(len(sentences)):
    sentence=sentences[j]

    input_id=tokenizer.encode(sentence, return_tensors="pt")

    outputs = model.forward(input_ids=input_id)

    logit = outputs.logits

    prob = F.softmax(logit, dim=-1)

    sum_prob=[]
    for i in tqdm(range(1,len(prob[0]))):
        p=prob[0,i,input_id[0,i]]
        sum_prob.append(p)

    sorted_prob=sorted(sum_prob)
    k=9 #超参数
    min_k=len(sentence)*k//100

    selected_prob=sorted_prob[:min_k]

    log_probs=[]

    for i in range(len(selected_prob)):
        log_probs.append(-1*math.log(selected_prob[i]))

    min_k_prob=np.mean(log_probs)

    sum_min_k_prob.append(min_k_prob)

with open('sum_min_k_prob.json', 'w') as f:
    json.dump(sum_min_k_prob, f)

print("success!")


100%|██████████| 860/860 [00:00<00:00, 61932.99it/s]


17.514509683675634
